# Test grainmaps.py
## Mainly to test reading tensor maps and merging voxels that have close misorientations, thus to generate 'labels'
## Jan 2025

In [ ]:
import os

os.environ['OMP_NUM_THREADS'] = '1'
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'

exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())
PYTHONPATH = setup_ImageD11_from_git( ) # ( os.path.join( os.environ['HOME'],'Code'), 'ImageD11_git' )

In [ ]:
# import functions we need

import concurrent.futures

# %matplotlib ipympl

import h5py
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
from skimage.filters import threshold_otsu
from skimage.morphology import convex_hull_image

import ImageD11.columnfile
from ImageD11.grain import grain
from ImageD11.peakselect import select_ring_peaks_by_intensity
from ImageD11.sinograms.sinogram import GrainSinogram, build_slice_arrays, write_slice_recon, read_h5, write_h5, get_2d_peaks_from_4d_peaks
from ImageD11.sinograms.roi_iradon import run_iradon
from ImageD11.sinograms.tensor_map import TensorMap
import ImageD11.sinograms.dataset
import ImageD11.nbGui.nb_utils as utils

import ipywidgets as widgets
from ipywidgets import interact

# test reading the pbp_tensormap_refined.h5

In [ ]:
from ImageD11.forward_model import io

In [ ]:
from ImageD11.forward_model import grainmaps

In [ ]:
# filename='/data/visitor/ma6288/id11/20241119/PROCESSED_DATA/A2050_DN_W340_nscope/A2050_DN_W340_nscope_full_slice/A2050_DN_W340_nscope_full_slice_grains.h5'
# filename = '/data/visitor/ma6288/id11/20241119/PROCESSED_DATA/A2050_W370_nscope/A2050_W370_nscope_full_slice/pbp_tensormap_refined.h5'
filename = '/data/visitor/ma6288/id11/20241119/PROCESSED_DATA/A2050_DN_W340_nscope_5pct_strained/A2050_DN_W340_nscope_5pct_strained_full_slice/pbp_tensormap_refined.h5'

In [ ]:
gm = grainmaps.grainmap(filename = filename)

In [ ]:
# check the 'labels' before merging
plt.figure()
plt.imshow(gm.DS['labels'][0,:,:])

# Do merging

In [ ]:
# merging regions using default settings, i.e. min_misori = 3, dis_tol = np.sqrt(2)
gm.merge_and_identify_grains()

In [ ]:
plt.figure()
img = plt.imshow(gm.DS['labels'][0, :, :], cmap='viridis')  # You can specify a colormap
plt.colorbar(img, label='grain ID')

In [ ]:
# further merge with a bigger mis_misori and dis_tol
gm.min_misori = 5
gm.merge_and_identify_grains(dis_tol=np.sqrt(3))

In [ ]:
plt.figure()
img = plt.imshow(gm.DS['labels'][0, :, :], cmap='viridis')  # You can specify a colormap
plt.colorbar(img, label='grain ID')

In [ ]:
# remove small grains that are considered as noise
DS_new = grainmaps.DS_remove_small_grains(gm.DS, min_vol=5)

In [ ]:
plt.figure()
img = plt.imshow(DS_new['labels'][0, :, :], cmap='viridis')  # You can specify a colormap
plt.colorbar(img, label='grain ID')

In [ ]:
# update the DS in gm class
gm.DS = DS_new

In [ ]:
gm.outname

In [ ]:
# to save the DS as gm.outname, by default DS.h5
gm.write_DS()

In [ ]:
# to read the newly merged grain map file
DS0 = io.read_h5_file(gm.outname)

In [ ]:
plt.figure()
img = plt.imshow(DS0['labels'][0, :, :], cmap='viridis')  # You can specify a colormap
plt.colorbar(img, label='grain ID')